# Setup

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import gc
import math
import random
from glob import glob
from collections import defaultdict, Counter
from typing import List, Dict
import joblib
import pickle

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import polars as pl
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import GroupKFold

## constants

In [4]:
EXP_NAME = "exp123"
DIR = "/gdrive/MyDrive/amazon_kdd_2023/"
K_FOLDS = 3
SEED = 42
LOCALES = ["UK", "JP", "DE"]

# This parameter controls to which end item the candidate is tied. 
# For example, if [1,2], candidates are generated from the last item and second last item in each session.
LAST_NS = [1, 2, 3] 

In [5]:
USE_FEATURES = [
    # === candidate features ===
    "co_visit_weight_last1", "consective_1_weight_last1", "consective_3_weight_last1", "consective_5_weight_last1", "similarity_score_last1", "bert_distance_last1", "lift_last1", "prone_distance_last1",
    "co_visit_weight_last2", "consective_1_weight_last2", "consective_3_weight_last2", "consective_5_weight_last2", "similarity_score_last2", "bert_distance_last2", "lift_last2", "prone_distance_last2",
    "co_visit_weight_last3", "consective_1_weight_last3", "consective_3_weight_last3", "consective_5_weight_last3", "similarity_score_last3", "bert_distance_last3", "lift_last3", "prone_distance_last3",
    "imf_score", "bpr_score",
    "co_visit_rank_last1", "consective_1_rank_last1", "consective_3_rank_last1", "consective_5_rank_last1", "similarity_rank_last1", "bert_rank_last1", "lift_rank_last1", "prone_rank_last1",
    "co_visit_rank_last2", "consective_1_rank_last2", "consective_3_rank_last2", "consective_5_rank_last2", "similarity_rank_last2", "bert_rank_last2", "lift_rank_last2", "prone_rank_last2",
    "co_visit_rank_last3", "consective_1_rank_last3", "consective_3_rank_last3", "consective_5_rank_last3", "similarity_rank_last3", "bert_rank_last3", "lift_rank_last3", "prone_rank_last3",
    "imf_rank", "bpr_rank",
    # === session features ===
    "S_session_length",
    "S_nunique_brand",
    "S_ratio_unique_brand",
    "S_nunique_item",
    "S_ratio_repurchase",
    "S_locale",
    "S_mean_price", "S_max_price", "S_min_price", "S_std_price", "S_total_amount",
    "S_color_not_null_count", "S_size_not_null_count", "S_model_not_null_count", "S_material_not_null_count", "S_author_not_null_count",
    "S_last_item_price",
    # === product features ===
    "P_price",
    "P_purchase_count", "P_purchase_count_global",
    "P_total_amount",
    "P_brand_purchase_count", "P_brand_purchase_count_global",
    "P_brand_mean_price", "P_brand_max_price", "P_brand_min_price", "P_brand_std_price", "P_total_brand_amount",
    "P_price_diff_to_avg_brand_price",
    "P_n_unique_locale",
    "P_is_color_null", "P_is_size_null", "P_is_model_null", "P_is_material_null", "P_is_author_null",
    "P_purchase_count_ratio_to_locale", "P_purchase_amount_ratio_to_locale", "P_purchase_count_ratio_to_brand", "P_purchase_amount_ratio_to_brand",
    # === session * product features ===
    "SP_price_diff_to_mean_price", "SP_price_diff_to_min_price", "SP_price_diff_to_max_price", "SP_price_diff_to_last_price",
    "SP_brand_price_diff_to_mean_price", "SP_brand_price_diff_to_min_price", "SP_brand_price_diff_to_max_price", "SP_brand_price_diff_to_last_price",
    "SP_same_brand_last1", "SP_same_brand_last2", "SP_same_brand_last3",
    "SP_same_color_last1", "SP_same_color_last2", "SP_same_color_last3",
    "SP_same_size_last1", "SP_same_size_last2", "SP_same_size_last3",
    "SP_same_model_last1", "SP_same_model_last2", "SP_same_model_last3",
    "SP_same_material_last1", "SP_same_material_last2", "SP_same_material_last3",
    "SP_same_author_last1", "SP_same_author_last2", "SP_same_author_last3",
    "SP_same_brand_sum", "SP_same_color_sum", "SP_same_size_sum", "SP_same_model_sum", "SP_same_material_sum", "SP_same_author_sum",
    # === similality features ===
    "imf_similarity", "bpr_similarity",
    "graph_emb_similarity_last1", "graph_emb_similarity_last2", "graph_emb_similarity_last3", 
    "i2v_similarity_last1", "i2v_similarity_last2", "i2v_similarity_last3", 
]

## functions

In [6]:
# functions for model training, prediction and evaluation

def predict(df:pl.DataFrame, chunk_size:int=30_000_000) -> np.array:
    preds = np.zeros((df.shape[0],))
    for fold in range(K_FOLDS):
        # load model
        model = pickle.load(open(f'{DIR}models/task1/{EXP_NAME}_{fold+1}.pkl', "rb"))
        # chunk data and predict to prevent OOM
        preds_by_one_model = []
        for frame in df.iter_slices(n_rows=chunk_size):
            preds_chunk = model.predict(frame[USE_FEATURES].to_pandas(), num_iteration=model.best_iteration)
            preds_by_one_model.append(preds_chunk)
        preds += np.concatenate(preds_by_one_model)
    preds /= K_FOLDS
    return preds


def rr_at_k(user_relevances:List[int], k:int=100) -> float:
    user_relevances = user_relevances[:k]
    rr = 0
    for i, label in enumerate(user_relevances):
        if label == 1:
            rr = 1/(i+1)
            break
    return rr

def mrr_at_k(users_relevances:List[List[int]], k:int=100) -> float:
    rr = 0
    for user_relevances in users_relevances:
        rr += rr_at_k(user_relevances, k)
    mrr = rr / len(users_relevances)
    return mrr

## fix seed

In [7]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
seed_everything(SEED)

# Train and Inference

## inference

In [8]:
files = glob(DIR + f"data/interim/for_ranker/task1/test_chunk_exp115_*")

dfs = []
for f in tqdm(files):
    print(f)
    df = pl.read_parquet(f)

    # prediction
    df = df.with_columns(
        pl.Series(name="pred", values=predict(df))
    )
    df = df[["session_id", "locale", "candidate_item", "pred"]]
    df = df.sort(["session_id", "pred"], descending=[False, True])
    df = df.groupby("session_id", maintain_order=True).head(100)
    dfs.append(df)

test = pl.concat(dfs)

  0%|          | 0/5 [00:00<?, ?it/s]

/gdrive/MyDrive/amazon_kdd_2023/data/interim/for_ranker/task1/test_chunk_exp115_0


 20%|██        | 1/5 [16:55<1:07:42, 1015.57s/it]

/gdrive/MyDrive/amazon_kdd_2023/data/interim/for_ranker/task1/test_chunk_exp115_1


 40%|████      | 2/5 [33:37<50:22, 1007.40s/it]  

...

 60%|██████    | 3/5 [51:08<34:15, 1027.61s/it]

/gdrive/MyDrive/amazon_kdd_2023/data/interim/for_ranker/task1/test_chunk_exp115_3


 80%|████████  | 4/5 [1:08:23<17:10, 1030.37s/it]

/gdrive/MyDrive/amazon_kdd_2023/data/interim/for_ranker/task1/test_chunk_exp115_4


100%|██████████| 5/5 [1:26:01<00:00, 1032.23s/it]


In [9]:
test = test.with_columns(
    pl.col("session_id").str.slice(12).cast(pl.Int32).alias("session_id")
)

In [10]:
test.write_parquet(DIR + "data/output/task1/" + EXP_NAME + "_1" + ".parquet", use_pyarrow=True)